## Pydantic Default Factory


#### 1. The "Static" Trap vs. The Solution


In [1]:
from datetime import datetime
from pydantic import BaseModel, Field
import time


# ❌ THE BAD WAY (Don't do this)
# 'created_at' is fixed to the time the class was DEFINED, not when the object is created.
class BadLog(BaseModel):
    message: str
    created_at: datetime = datetime.now()


# ✅ THE GOOD WAY (Do this)
# 'default_factory' accepts a callable (the function itself, not the result).
# It runs 'datetime.now' every time a new instance is created.
class GoodLog(BaseModel):
    message: str
    created_at: datetime = Field(default_factory=datetime.now)

In [2]:
print("--- The Problem ---")
log1 = BadLog(message="First log")
time.sleep(2)  # Wait 2 seconds
log2 = BadLog(message="Second log")

print(f"Log 1 Time: {log1.created_at}")
print(f"Log 2 Time: {log2.created_at}")

--- The Problem ---
Log 1 Time: 2025-12-10 21:40:28.380908
Log 2 Time: 2025-12-10 21:40:28.380908


In [3]:
print("--- The Solution ---")
log3 = GoodLog(message="First log")
time.sleep(2)  # Wait 2 seconds
log4 = GoodLog(message="Second log")

print(f"Log 3 Time: {log3.created_at}")
print(f"Log 4 Time: {log4.created_at}")

--- The Solution ---
Log 3 Time: 2025-12-10 21:41:12.499131
Log 4 Time: 2025-12-10 21:41:14.504318


---

#### 2. Generating Unique IDs (UUIDs)


In [4]:
from uuid import UUID, uuid4
from pydantic import BaseModel, Field


class BadUser(BaseModel):
    id: UUID = uuid4()
    username: str


class GoodUser(BaseModel):
    # We pass 'uuid4' without parentheses.
    # Pydantic calls uuid4() for us when we make a new User.
    id: UUID = Field(default_factory=uuid4)
    username: str

In [5]:
user_a = BadUser(username="yash_dev")
user_b = BadUser(username="yash_coder")

print(f"User A ID: {user_a.id}")
print(f"User B ID: {user_b.id}")

User A ID: 9d15d313-e5d5-4730-8975-f975e792e438
User B ID: 9d15d313-e5d5-4730-8975-f975e792e438


In [6]:
user_a = GoodUser(username="yash_dev")
user_b = GoodUser(username="yash_coder")

print(f"User A ID: {user_a.id}")
print(f"User B ID: {user_b.id}")

User A ID: 4ad1acec-62bb-40d5-a55a-a613009d5b04
User B ID: 14e75a21-9cb3-470a-990f-fd77525810b1


#### 3. Advanced Custom Logic


In [7]:
import random
import string
from pydantic import BaseModel, Field


# A custom function to generate a 6-character alphanumeric code
def generate_sku():
    chars = string.ascii_uppercase + string.digits
    return "".join(random.choices(chars, k=6))


class BadProduct(BaseModel):
    name: str
    price: float

    sku: str = generate_sku()


class GoodProduct(BaseModel):
    name: str
    price: float
    # Uses our custom function
    sku: str = Field(default_factory=generate_sku)

In [8]:
p1 = BadProduct(name="Gaming Mouse", price=59.99)
p2 = BadProduct(name="Mechanical Keyboard", price=120.00)

print(f"Product: {p1.name}, SKU: {p1.sku}")
print(f"Product: {p2.name}, SKU: {p2.sku}")

Product: Gaming Mouse, SKU: 0JXE8S
Product: Mechanical Keyboard, SKU: 0JXE8S


In [9]:
p1 = GoodProduct(name="Gaming Mouse", price=59.99)
p2 = GoodProduct(name="Mechanical Keyboard", price=120.00)

print(f"Product: {p1.name}, SKU: {p1.sku}")
print(f"Product: {p2.name}, SKU: {p2.sku}")

Product: Gaming Mouse, SKU: 8XRONN
Product: Mechanical Keyboard, SKU: QKYMJJ
